# Convert hex to base64

In [2]:
string_to_convert = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

In [3]:
import binascii, base64
base64.b64encode(binascii.unhexlify(string_to_convert)).decode()

'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

# Fixed XOR

In [4]:
cipher = "1c0111001f010100061a024b53535009181c"

In [5]:
xor_block = "686974207468652062756c6c277320657965"

In [7]:
!pip install numpy
import numpy

In [ ]:
def xor_encrypt(cipher: bytes, block: bytes):
    pass